In [ ]:
1. init data
2. train vae
3. use vae as wrapper for env
4. make other wrappers, prepro, frame stack, FRAME CAPTURE
5. train rl, gather ne imgs with capture wrapper ( also capture actions for rl)
6. if n of new img == num of old imgs
    Retrain vae
    else continue rl training and img gathering
7. continou train vae
8. performance check of rl:
    if better or == to previous performance
    continou training that rl and collecting imgs with it
    else train completle new rl as often until its better than previous.. 
    if it doesnt work
    


In [66]:
import time
import os
import torch
from torchvision import transforms
import random
import numpy as np
import yaml
from PIL import Image
from experiment import VAEXperiment
from models import *


import gymnasium as gym
from gymnasium import ObservationWrapper
from gymnasium.wrappers import PixelObservationWrapper, FrameStack
from gymnasium.spaces import Box, Discrete



from stable_baselines3 import SAC, PPO, A2C
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.callbacks import CallbackList, CheckpointCallback, EvalCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy

# Create ENV

## 1 Collect pictures with random Agent

In [ ]:
#Variables
save_path =
num_of_episodes_to_collect=


class frame_saver(ObservationWrapper):
    def __init__(self, env,
                 collect_frames_dir = None,
                 start_index = 0):
        super().__init__(env)
        
        self.collect_frames_dir = collect_frames_dir
        self.frame_idx = start_index
                
        
    def observation(self, obs):
        frame = obs['pixels']#.to('cuda')
        if self.collect_frames_dir != None:
            im = Image.fromarray(np.array(frame))
            im.save(self.collect_frames_dir+'_'+str(self.frame_idx)+'.jpeg')
            self.frame_idx += 1
        return obs

#Environment
init_env = gym.make("MountainCarContinuous-v0",
                render_mode ='rgb_array')
seed = 42
init_env.reset(seed=seed)
init_env = PixelObservationWrapper(env)
init_env = frame_saver(env, save_path)
init_env = Monitor(env)

#Agent
agent = A2C(

    env = env,
    n_steps= 100,
    
    policy='MlpPolicy',
    ent_coef= 0.0,
    use_sde=True,
    sde_sample_freq = 16,
    policy_kwargs= dict(log_std_init=0.0, ortho_init=False))

#collect images
mean_reward, std_reward = evaluate_policy(agent, eval_env, n_eval_episodes=2)

## 2 Train first VAE and make latent obs environment for the Agent

## 3 Double loop
### Outer loop: continue training of VAE if n of new pictures = n of old pictures or performance drop of RL-agent
### Inner loop: train RL agent while collecting images
### Breaks: if RL agent gets stuck continue train of VAE, if still stuck, retrain RL from scratch, if still stuck, retrain VAE from scratch
### if RL agent is trained from scratch, do imitation learning with past trajectories

In [2]:
from gymnasium import ObservationWrapper
from gymnasium.wrappers import PixelObservationWrapper, FrameStack
from gymnasium.spaces import Box, Discrete
import numpy as np
import yaml
from experiment import VAEXperiment
from models import *
import torch
from torchvision import transforms
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv

class AddGaussianNoise(object):
    def __init__(self, mean=0., std=0.1):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

#ideas
#maybe use wrapper to catch and save frames during RL model learning
#alternative is to use the model afterwards to generate frames during a test run
#
#??return mu, std or sample or just mu??
#
class VAE_ENC(ObservationWrapper):
    def __init__(self, env, vae, latent_dim,
                 mean=0,std=0.1,
                 size=(64,64),
                 collect_frames_dir = None,
                 start_index = 0):
        super().__init__(env)
        #new obs space with std
        #self.observation_space = Box(shape=(2, latent_dim), low=-np.inf, high=np.inf)
        #just mean
        self.observation_space = Box(shape=(latent_dim,), low=-np.inf, high=np.inf)
        
        self.vae = vae
        #transforms
        self.mean = mean
        self.std = std
        self.size = size
        
        self.collect_frames_dir = collect_frames_dir
        self.frame_idx = start_index
        
        
        
    def observation(self, obs):
        #get frame
        #print(obs)
        frame = obs['pixels']#.to('cuda')
        if self.collect_frames_dir == 1:
            im = Image.fromarray(np.array(frame))
            im.save(self.collect_frames_dir+'_'+str(self.frame_idx)+'.jpeg')
            self.frame_idx += 1
        #transform for VAE
        val_transforms = transforms.Compose([transforms.ToTensor(),
        #transforms.RandomHorizontalFlip(),
        AddGaussianNoise(self.mean, self.std),
        transforms.Resize(self.size),
        #transforms.Grayscale(),
        #transforms.Normalize(self.mean, self.std),
        ])
        frame = val_transforms(frame) #(c,h,w)
        frame = torch.unsqueeze(frame, 0)#.to(self.device) #make it (1,c,h,w)
        enc = self.vae.encode(frame)    
        enc = np.array([tensor.detach().cpu().numpy() for tensor in enc])
        #with std
        #enc = np.array([enc[0][0], enc[1][0]]) ## mu, std #  give only mu?
        #just mean
        enc = np.array(enc[0][0])
        

            
        
        
        return enc
    
def get_vae(version='version_0',log_directory='logs/BCE_sum_VAE/MSSIMVAE/'):

    model_path=log_directory+'/'+version+'/hparams.yaml'
    ckpt_path=log_directory+'/'+version+'/checkpoints/last.ckpt'

    config = yaml.safe_load(open(model_path))
    model = vae_models[config['model_params']['name']](**config['model_params'])
    ckpt = torch.load(ckpt_path)
    experiment = VAEXperiment(model, config['exp_params'])
    experiment.load_state_dict(ckpt['state_dict'])      
    vae = experiment.model
    return vae

In [12]:
data_name='wrapper_test'
save_path='Data/MountainCar/'+data_name+'/'

vae = get_vae(version='version_13',log_directory='logs/MountainCar/BCE_sum_VAE_2/MSSIMVAE/')

In [187]:
env = gym.make("MountainCarContinuous-v0",
                render_mode ='rgb_array')
seed = 42
env.reset(seed=seed)
env = Monitor(env)
env = PixelObservationWrapper(env)
env = frame_saver(env, save_path)
env = VAE_ENC(env, vae, 2, collect_frames_dir = save_path)
env = FrameStack(env, num_stack=2, lz4_compress = False)



In [196]:
def eval_agent(agent, env, n_eval_episodes):
    total_rewards = []
    device = 'cpu'
    policy = agent.policy.to(device)  # Move observation to the same device as the model
    print(policy)
    for episode in range(n_eval_episodes):
        observation, info = env.reset()
        episode_reward = 0
        done = False
        print(observation)
        while not done: 
            action = policy.predict(observation, deterministic=True)
            #print(action)
            #print(action[0])
            #action = action[0]
            #action= env.action_space.sample()
            #print(action)
            observation, reward, terminated, truncated, info = env.step(action)
            print(observation[0],observation[1])
            if terminated:
                done = True
            if truncated:
                done = True
        total_rewards.append(episode_reward)

    total_rewards = np.array(total_rewards)
    return np.mean(total_rewards), np.std(total_rewards)

In [197]:
obs, info = env.reset()
device = 'cpu'
action= env.action_space.sample()
print(action)

[0.8700861]


In [198]:
env.observation_space

Box(-inf, inf, (2, 2), float32)

In [199]:
eval_agent(agent, env, 1)


ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=4, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=4, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=1, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)
[-59.589066  22.560253] [-59.60724   22.332514]
[-59.60724   22.332514] [-59.68515   22.114922]
[-59.68515   22.114922] [-59.476707  2

[-56.829624  30.844378] [-57.10473   30.478308]
[-57.10473   30.478308] [-57.270885  30.048874]
[-57.270885  30.048874] [-57.575104  29.685997]
[-57.575104  29.685997] [-57.493347  29.512205]
[-57.493347  29.512205] [-57.495743  29.331839]
[-57.495743  29.331839] [-56.135685  31.83848 ]
[-56.135685  31.83848 ] [-56.356293  31.849152]
[-56.356293  31.849152] [-55.375698  32.732117]
[-55.375698  32.732117] [-55.228333  33.143673]
[-55.228333  33.143673] [-55.117268  33.12866 ]
[-55.117268  33.12866 ] [-54.285458  34.842297]
[-54.285458  34.842297] [-53.765034  35.904198]
[-53.765034  35.904198] [-53.644234  36.210594]
[-53.644234  36.210594] [-52.84314  37.92459]
[-52.84314  37.92459] [-52.88978   37.581795]
[-52.88978   37.581795] [-52.940083  37.547295]
[-52.940083  37.547295] [-51.24386   39.607586]
[-51.24386   39.607586] [-50.373478  40.183926]
[-50.373478  40.183926] [-49.955933  40.526917]
[-49.955933  40.526917] [-47.299862  43.30856 ]
[-47.299862  43.30856 ] [-47.064342  43.6903

[-41.481773  48.24748 ] [-43.76827   45.883045]
[-43.76827   45.883045] [-44.8067    44.927265]
[-44.8067    44.927265] [-44.66397   44.889027]
[-44.66397   44.889027] [-45.892223  43.904457]
[-45.892223  43.904457] [-46.874104  43.48414 ]
[-46.874104  43.48414 ] [-46.756916  43.678368]
[-46.756916  43.678368] [-46.94622   43.469975]
[-46.94622   43.469975] [-46.973446  43.426018]
[-46.973446  43.426018] [-47.283466  42.94975 ]
[-47.283466  42.94975 ] [-48.063904  42.31581 ]
[-48.063904  42.31581 ] [-49.927124  40.461308]
[-49.927124  40.461308] [-50.170567  40.266125]
[-50.170567  40.266125] [-51.004982  39.682415]
[-51.004982  39.682415] [-51.928646  38.790974]
[-51.928646  38.790974] [-52.84299   37.756214]
[-52.84299   37.756214] [-52.877502  37.691967]
[-52.877502  37.691967] [-52.782433  37.68349 ]
[-52.782433  37.68349 ] [-52.894627  37.596752]
[-52.894627  37.596752] [-53.329613  36.9072  ]
[-53.329613  36.9072  ] [-53.610245  36.25788 ]
[-53.610245  36.25788 ] [-53.66859  35.7

[-50.942802  39.62712 ] [-50.059364  40.289574]
[-50.059364  40.289574] [-50.213314  40.34525 ]
[-50.213314  40.34525 ] [-48.383183  42.011295]
[-48.383183  42.011295] [-46.945267  43.472683]
[-46.945267  43.472683] [-46.73933  43.58726]
[-46.73933  43.58726] [-46.88308   43.526543]
[-46.88308   43.526543] [-46.866173  43.598354]
[-46.866173  43.598354] [-46.7858    43.532204]
[-46.7858    43.532204] [-46.962376  43.334297]
[-46.962376  43.334297] [-45.982735  44.00653 ]
[-45.982735  44.00653 ] [-44.523357  45.12584 ]
[-44.523357  45.12584 ] [-44.66762   44.948475]
[-44.66762   44.948475] [-45.021103  44.39561 ]
[-45.021103  44.39561 ] [-43.816753  45.795788]
[-43.816753  45.795788] [-43.698555  45.801727]
[-43.698555  45.801727] [-41.4535   48.20463]
[-41.4535   48.20463] [-41.85078  47.92205]
[-41.85078  47.92205] [-41.71304   47.887047]
[-41.71304   47.887047] [-41.057888  48.450703]
[-41.057888  48.450703] [-41.056812  48.3272  ]
[-41.056812  48.3272  ] [-40.780663  48.635605]
[-40

[-46.791115  43.555553] [-47.17996   43.107307]
[-47.17996   43.107307] [-47.26101   43.113693]
[-47.26101   43.113693] [-48.318615  41.952995]
[-48.318615  41.952995] [-49.664516  40.688084]
[-49.664516  40.688084] [-50.001686  40.33867 ]
[-50.001686  40.33867 ] [-50.054424  40.415356]
[-50.054424  40.415356] [-50.406624  40.02965 ]
[-50.406624  40.02965 ] [-50.880054  39.88282 ]
[-50.880054  39.88282 ] [-50.592278  40.078106]
[-50.592278  40.078106] [-51.77019   39.100548]
[-51.77019   39.100548] [-51.78974   39.050873]
[-51.78974   39.050873] [-52.654175  37.908752]
[-52.654175  37.908752] [-52.9345   37.67802]
[-52.9345   37.67802] [-52.859104  37.427082]
[-52.859104  37.427082] [-52.815514  37.6999  ]
[-52.815514  37.6999  ] [-52.84008   37.709118]
[-52.84008   37.709118] [-52.80093  37.73406]
[-52.80093  37.73406] [-53.082684  37.503822]
[-53.082684  37.503822] [-52.73961   37.741383]
[-52.73961   37.741383] [-52.832077  37.63482 ]
[-52.832077  37.63482 ] [-52.75606   37.742958]


[-49.967487  40.45986 ] [-48.105534  42.324768]
[-48.105534  42.324768] [-47.26415  43.0388 ]
[-47.26415  43.0388 ] [-47.110077  43.17792 ]
[-47.110077  43.17792 ] [-47.019928  43.577885]
[-47.019928  43.577885] [-47.042538  43.215145]
[-47.042538  43.215145] [-46.962223  43.517384]
[-46.962223  43.517384] [-46.873882  43.770657]
[-46.873882  43.770657] [-46.78392  43.71177]
[-46.78392  43.71177] [-46.97293   43.458416]
[-46.97293   43.458416] [-47.091923  43.326927]
[-47.091923  43.326927] [-46.909416  43.54411 ]
[-46.909416  43.54411 ] [-46.69951  43.67365]
[-46.69951  43.67365] [-46.652225  43.605686]
[-46.652225  43.605686] [-46.766125  43.526302]
[-46.766125  43.526302] [-46.40938   43.513294]
[-46.40938   43.513294] [-46.04905   43.790466]
[-46.04905   43.790466] [-44.5582   45.05916]
[-44.5582   45.05916] [-44.807976  44.760017]
[-44.807976  44.760017] [-45.013184  44.781326]
[-45.013184  44.781326] [-44.846554  44.77993 ]
[-44.846554  44.77993 ] [-44.746887  44.855354]
[-44.746

(0.0, 0.0)

In [200]:
a= env.action_space.sample()

In [203]:
env.reset()
env.step([0.5])

(<gymnasium.wrappers.frame_stack.LazyFrames at 0x7fe2b73bd040>,
 -0.025,
 False,
 False,
 {})

In [137]:
policy.predict(obs)

(array([-1.], dtype=float32), None)

In [94]:
from stable_baselines3.common.env_checker import check_env

check_env(env)

AssertionError: The observation returned by `reset()` method must be a numpy array

In [65]:
env.spec

EnvSpec(id='MountainCarContinuous-v0', entry_point='gymnasium.envs.classic_control.continuous_mountain_car:Continuous_MountainCarEnv', reward_threshold=90.0, nondeterministic=False, max_episode_steps=999, order_enforce=True, autoreset=False, disable_env_checker=False, apply_api_compatibility=False, kwargs={'render_mode': 'rgb_array'}, namespace=None, name='MountainCarContinuous', version=0, additional_wrappers=(WrapperSpec(name='PixelObservationWrapper', entry_point='gymnasium.wrappers.pixel_observation:PixelObservationWrapper', kwargs={'pixels_only': True, 'render_kwargs': None, 'pixel_keys': ('pixels',)}), WrapperSpec(name='frame_saver', entry_point='__main__:frame_saver', kwargs=None), WrapperSpec(name='VAE_ENC', entry_point='__main__:VAE_ENC', kwargs=None), WrapperSpec(name='FrameStack', entry_point='gymnasium.wrappers.frame_stack:FrameStack', kwargs={'num_stack': 2, 'lz4_compress': False})), vector_entry_point=None)

In [ ]:
%

In [69]:
class frame_saver(ObservationWrapper):
    def __init__(self, env,
                 collect_frames_dir = None,
                 start_index = 0):
        super().__init__(env)
        
        self.collect_frames_dir = collect_frames_dir
        self.frame_idx = start_index
                
        
    def observation(self, obs):
        frame = obs['pixels']#.to('cuda')
        if self.collect_frames_dir != None:
            im = Image.fromarray(np.array(frame))
            im.save(self.collect_frames_dir+'aaa_'+str(self.frame_idx)+'.jpeg')
            self.frame_idx += 1
        return obs

In [123]:

def make_env(env_id: str = "MountainCarContinuous-v0", rank: int = 0, seed: int = 42) -> Callable:
    def _init() -> gym.Env:
        data_name='wrapper_test'
        save_path='Data/MountainCar/'+data_name+'/train_env_'+str(rank)+'_'
        vae = get_vae(version='version_13',log_directory='logs/MountainCar/BCE_sum_VAE_2/MSSIMVAE/')
        
        env = gym.make(env_id,
                    render_mode ='rgb_array')
        seed = 42
        env.reset(seed=seed + rank)
        env = PixelObservationWrapper(env)
        env = frame_saver(env, save_path)
        env = VAE_ENC(env, vae, 2, collect_frames_dir = save_path)
        env = FrameStack(env, num_stack=2)
        env = Monitor(env)
        return env
    set_random_seed(seed)
    return _init

In [73]:
env_id = "MountainCarContinuous-v0"
num_cpu = 2  # Number of processes to use
# Create the vectorized environment
env = SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)])

In [124]:
env = DummyVecEnv([make_env(env_id, i) for i in range(num_cpu)])

In [77]:
env

In [78]:
check_env(env)

AssertionError: Your environment must inherit from the gymnasium.Env class cf. https://gymnasium.farama.org/api/env/

In [127]:

# Tuned
agent = A2C(
    verbose=1,
    env = env,
    n_steps= 100,
    
    policy='MlpPolicy',
    ent_coef= 0.0,
    use_sde=True,
    sde_sample_freq = 16,
    policy_kwargs= dict(log_std_init=0.0, ortho_init=False))

Using cuda device


In [45]:
RL_train_steps = 1000
agent.learn(total_timesteps=RL_train_steps)

In [8]:
_env = make_env(env_id, 0)()

In [11]:
_env.spec

EnvSpec(id='MountainCarContinuous-v0', entry_point='gymnasium.envs.classic_control.continuous_mountain_car:Continuous_MountainCarEnv', reward_threshold=90.0, nondeterministic=False, max_episode_steps=999, order_enforce=True, autoreset=False, disable_env_checker=False, apply_api_compatibility=False, kwargs={'render_mode': 'rgb_array'}, namespace=None, name='MountainCarContinuous', version=0, additional_wrappers=(WrapperSpec(name='PixelObservationWrapper', entry_point='gymnasium.wrappers.pixel_observation:PixelObservationWrapper', kwargs={'pixels_only': True, 'render_kwargs': None, 'pixel_keys': ('pixels',)}), WrapperSpec(name='frame_saver', entry_point='__main__:frame_saver', kwargs=None), WrapperSpec(name='VAE_ENC', entry_point='__main__:VAE_ENC', kwargs=None), WrapperSpec(name='FrameStack', entry_point='gymnasium.wrappers.frame_stack:FrameStack', kwargs={'num_stack': 2, 'lz4_compress': False}), WrapperSpec(name='Monitor', entry_point='stable_baselines3.common.monitor:Monitor', kwargs=

In [20]:
eval_env = _env()

In [53]:
from stable_baselines3.common.env_checker import check_env

check_env(env)

/home/max/miniconda3/envs/vae/lib/python3.9/site-packages/stable_baselines3/common/env_checker.py:238: UserWarning: Your observation  has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


AssertionError: The observation returned by `reset()` method must be a numpy array

In [46]:
evaluate_policy(agent,env, n_eval_episodes=1)

(-74.850105, 0.0)

In [21]:
env_id = "MountainCarContinuous-v0"
num_cpu = 2 
eval_env = SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)])

In [22]:
mean_reward, std_reward = evaluate_policy(agent, eval_env, n_eval_episodes=2)

In [23]:
mean_reward

-33.7861745

In [12]:
# Find latest
import os, glob
directory = "logs/MountainCar/BCE_sum_VAE_1/MSSIMVAE"
b = max(glob.glob(os.path.join(directory, '*/')), key=os.path.getmtime)

In [13]:
b

'logs/MountainCar/BCE_sum_VAE_1/MSSIMVAE/version_4/'